# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [3]:
!pip install -U torch==1.10

In [4]:
ls

sample_data/


In [ ]:
!git clone https://github.com/psolikov/TTS_HW
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz


In [ ]:
# ! curl gdrive.sh | bash -s https://drive.google.com/file/d/1XIUvH0QXcZ6Ev0DbNSPGJIdC3-h03Sx9/view?usp=sharing
!!gdown --id 1XIUvH0QXcZ6Ev0DbNSPGJIdC3-h03Sx9

In [ ]:
!unzip file.zip -d down_file/


In [8]:
!cp -r down_file/content/TTS_HW/libri_training_data/ /content/TTS_HW/libri_training_data/

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [ ]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

In [ ]:
% cd apex
! git reset --hard a651e2c
% cd ..

/content/apex
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/content


In [ ]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

In [ ]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

nvidia_fastpitch_210824.pt already downloaded.
nvidia_waveglow256pyt_fp16.pt already downloaded.


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [ ]:
ls /content/output

ls: cannot access '/content/output': No such file or directory


In [ ]:
# !mkdir /content/output/modules_inputs/

In [ ]:
!ls 

apex	   file.zip		      input-from-train.pt  sample_data
down_file  input-from-train-batch.pt  output		   TTS_HW


In [ ]:
!ls /content/output/modules_inputs

In [ ]:
! ls /content/output/modules_inputs

In [ ]:
!cd TTS_HW && bash scripts/train.sh

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [ ]:
# !cd TTS_HW && bash scripts/inference_example.sh

In [ ]:
import IPython
IPython.display.Audio('/content/output/audio_phrase_1_64.txt/audio_0.wav')

ValueError: ignored

In [ ]:
import IPython
IPython.display.Audio('/content/output/audio_phrase_1_64.txt/audio_0.wav')

ValueError: ignored

In [ ]:
1+3

4

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [ ]:
pip install transformers torchmetrics

In [1]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics
!pip install seqeval

import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device='cuda:0'

  Using cached DLLogger-0.1.0-py3-none-any.whl


In [ ]:
ls TTS_HW/libri_training_data

In [ ]:
!tail -n 50 "TTS_HW/libri_training_data/40_121026_000178_000000.TextGrid"


In [22]:
!tail "TTS_HW/libri_training_data/40_121026_000178_000000.prom"

40_121026_000178_000000	7.250	7.700	gaze	0.611	0.769
40_121026_000178_000000	7.700	7.820	and	0.347	0.000
40_121026_000178_000000	7.820	8.380	contracted	0.882	0.360
40_121026_000178_000000	8.380	9.020	features	0.732	1.842
40_121026_000178_000000	9.340	9.760	dumb	1.672	0.135
40_121026_000178_000000	9.760	9.990	and	0.775	0.000
40_121026_000178_000000	9.990	10.620	motionless	0.237	1.103
40_121026_000178_000000	10.620	10.760	as	0.205	0.000
40_121026_000178_000000	10.760	10.830	a	0.094	0.006
40_121026_000178_000000	10.830	11.330	statue	1.124	1.000


In [31]:
" ".join(["a", "b", "c"])

'a b c'

In [2]:
sentences = []
label_proms = []
label_boundaries = []
for filename in tqdm(Path('TTS_HW/libri_training_data').glob('*.prom')):
    #####################
    ##### YOUR CODE #####
    #####################
    # pass
    with open(filename, "r") as f:
      sentence = []
      label_prom_s = []
      for line in f.readlines():
        x = float(line.split()[-2])
        y = float(line.split()[-1])
        sentence.append(line.split()[3])
        label_prom_s.append(y)
      label_proms.append(label_prom_s)
      sentences.append(" ".join(sentence))

341it [00:00, 10481.15it/s]


In [3]:
torch.cuda.empty_cache()

In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    flat_preds = np.argmax(preds, axis=2).flatten()
    flat_labels = labels.flatten()
    return np.sum(flat_preds == flat_labels)/len(flat_labels)

In [4]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3)
acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = [[val_to_class(label) for label in label_s] for label_s in labels]
        self.tokenizer = tokenizer
        self.max_len = max_len
        # self.texts = [tokenizer(text, 
        #                        padding='max_length', max_length = 512, truncation=True,
        #                         return_tensors="pt") for text in self.sentences]
        
    def __getitem__(self, index):
        #####################
        ##### YOUR CODE #####
        #####################
        # input_token = self.texts[index]
        sentence = str(self.sentences[index])
        input_token = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = input_token["input_ids"]
        mask = input_token["attention_mask"]
        label = self.labels[index]
        label.extend([0]*200)
        label=label[:200]
        # print(f"mask len: {len(mask[0])}, label len: {len(label)}")
        # label += [-1] * (len(mask[0]) - len(label))
        # print((mask.shape[0] - len(label)))
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in tqdm(Path('TTS_HW/libri_training_data').glob('*.prom')):
    #####################
    ##### YOUR CODE #####
    #####################
    # pass
    with open(filename, "r") as f:
      sentence = []
      label_prom_s = []
      for line in f.readlines():
        x = float(line.split()[-2])
        y = float(line.split()[-1])
        sentence.append(line.split()[3])
        label_prom_s.append(y)
      label_proms.append(label_prom_s)
      sentences.append(" ".join(sentence))

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

# training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
training_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=True)


model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=3)##### YOUR CODE #####
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    accs = []
    f1_scores = []
    for batch in t_bar:
        optimizer.zero_grad()
        # print(batch)
        in_ids, mask, tags = batch["ids"].to(device, dtype = torch.long),\
         batch["mask"].to(device, dtype = torch.long), batch["tags"].to(device, dtype = torch.long)
        # tags += 1
        # print(batch)
        # labels = torch.tensor([1] * in_ids.size(1)).unsqueeze(0)
        # out = model(**in_ids, labels=mask)
        loss = model(in_ids, mask, labels = tags)[0]
        loss.backward()

        # loss = f1_criterion(out, tags)

        # acc = acc_criterion(out, tags)
        # accs.append(acc)
        # f1_scores.append(acc)
        #####################
        ##### YOUR CODE #####
        #####################
        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.step()
    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        # predictions , true_labels = [], []

        for batch in test_loader:
            #####################
            ##### YOUR CODE #####
            #####################
            predictions , true_labels = [], []

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['tags'].to(device, dtype = torch.long)

            output = model(ids, mask, labels=targets)
            logits = output[1]
            out = torch.softmax(logits, dim=2)
            _, out = torch.max(out, dim=2)
            out_t = out[mask.type(torch.bool)].reshape(torch.sum(mask))
            targets_t = targets[mask.type(torch.bool)].reshape(torch.sum(mask))
            f1 = f1_criterion(out_t.to("cpu"), targets_t.to("cpu"))
            f1_scores.append(f1)
            # accs.append(torch.tensor(0.1))
            # print(out_t.shape, targets_t.shape)
            # loss, logits = output[:2]
            # logits_cpu = logits.detach().cpu().numpy()
            # label_ids = targets.to('cpu').numpy()
            # predictions.extend([list(p) for p in np.argmax(logits_cpu, axis=2)])
            # true_labels.append(label_ids)
            # # acc = acc_criterion(logits, targets)
            # # print(predictions, true_labels)
            # acc=0
            # # acc = acc_criterion(torch.tensor(predictions, dtype=torch.long), torch.tensor(true_labels, dtype=torch.long))
            # # eval_loss += loss.mean().item()
            # f1 = f1_criterion(torch.tensor(predictions[0], dtype=torch.long), torch.tensor(true_labels, dtype=torch.long))
            # f1_scores.append(f1)
            # accs.append(acc)


            # pass

    # print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

341it [00:00, 11121.06it/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model

EPOCH:0


  0%|          | 0/67 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
loss; 0.5625361204147339: 100%|██████████| 67/67 [00:27<00:00,

F1 score: 0.8153408765792847
EPOCH:1


loss; 0.387619286775589: 100%|██████████| 67/67 [00:27<00:00,  2.42it/s]


...

F1 score: 0.7843306660652161
EPOCH:8


loss; 0.004809824749827385: 100%|██████████| 67/67 [00:27<00:00,  2.42it/s]


F1 score: 0.7631248235702515
EPOCH:9


loss; 0.0017264668131247163: 100%|██████████| 67/67 [00:27<00:00,  2.43it/s]


F1 score: 0.7843306660652161


In [22]:
for x in test_dataset:
  print(x)
  break

{'ids': tensor([  101, 18257,  4208,  1163,  1677,  1465,  1103,  8228,  1377, 23373,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

In [ ]:
for x in train_dataset:
  print(x)
  breaK

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!

In [ ]:
!mkdir /content/output/models/

In [ ]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

train.py:46: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
Traceback (most recent call last):
  File "train.py", line 52, in <module>
    import amp_C
ImportError: /usr/local/lib/python3.7/dist-packages/amp_C.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_S2_
Killing subprocess 3257
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 340, in <module>
    main()
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 326, in main
    sigkill_handler(signal.SIGTERM, None)  # not coming back
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 301, in sigkill_handle

In [ ]:

! ls TTS_HW/output/FastPitch_checkpoint_[1].pt

'TTS_HW/output/FastPitch_checkpoint_[1].pt'


In [ ]:
!python TTS_HW/export_torchscript.py --generator-name FastPitch --generator-checkpoint TTS_HW/output/FastPitch_checkpoint_[1].pt

In [ ]:
1+2

3

In [ ]:
ls /content/output/models

FastPitch_trchscript_FastPitch.onnx


In [ ]:
!pip install torch==1.8

     |████████████████████████████████| 735.5 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.


In [ ]:
cat /usr/local/lib/python3.7/dist-packages/torch/_sources.py

import ast
import functools
import inspect
from textwrap import dedent
from typing import Any, Optional, Tuple, List, NamedTuple
from torch._C import ErrorReport
from torch._C._jit_tree_views import SourceRangeFactory

def get_source_lines_and_file(
    obj: Any,
    error_msg: Optional[str] = None,
) -> Tuple[List[str], int, Optional[str]]:
    """
    Wrapper around inspect.getsourcelines and inspect.getsourcefile.

    Returns: (sourcelines, file_lino, filename)
    """
    filename = None  # in case getsourcefile throws
    try:
        filename = inspect.getsourcefile(obj)
        sourcelines, file_lineno = inspect.getsourcelines(obj)
    except OSError as e:
        msg = (f"Can't get source for {obj}. TorchScript requires source access in "
               "order to carry out compilation, make sure original .py files are "
               "available.")
        if error_msg:
            msg += '\n' + error_msg
        raise OSError(msg) from e

    return sourcelines, file_lineno, 